In [1]:
import math
import os
import shutil
import time
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import torchvision.models as models
from HSI_class import HSI
import createSample as CS
import augmentation as aug

import simsiam.loader
import simsiam.builder

sample_per_class = 5
num_per_category_augment_1 = 10
num_per_category_augment_2 = 10
epochs = 200

random = 1

In [2]:
dataset_path = r"C:\Users\Asus TUF\Documents\code\TA\Hyperspectral oil spill detection datasets"

dataset = []

i = 0
for filename in os.listdir(dataset_path):
    if i > 1:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        dataset.append(hsi)
    i += 1

Processing file: C:\Users\Asus TUF\Documents\code\TA\Hyperspectral oil spill detection datasets\GM01.mat
Processing file: C:\Users\Asus TUF\Documents\code\TA\Hyperspectral oil spill detection datasets\GM02.mat


In [3]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

print(model_names)
# create model
arch = 'vgg16' 
print("=> creating model '{}'".format(arch))
model = simsiam.builder.SimSiam(
    models.__dict__[arch])

['alexnet', 'convnext_base', 'convnext_large', 'convnext_small', 'convnext_tiny', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_v2_l', 'efficientnet_v2_m', 'efficientnet_v2_s', 'get_model', 'get_model_builder', 'get_model_weights', 'get_weight', 'googlenet', 'inception_v3', 'list_models', 'maxvit_t', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf', 'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_128gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf', 'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext101_64x4d', 'resnext50_32x4d', 'shufflenet_v2_x0_

c:\Users\Asus TUF\Documents\code\TA\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Asus TUF\Documents\code\TA\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
hsi_ = dataset[0]
patch_size = 9
sample_per_class = sample_per_class

indices_0 = []
indices_1 = []

print(f"random: {random}")

if random:
    print("generating random sample")
    selected_patch_0, selected_patch_1, indices_0, indices_1 = CS.createSample(hsi_, patch_size, sample_per_class)
else:
    print("using generated indices")
    indices_0 = [(np.int64(45), np.int64(528)), (np.int64(872), np.int64(337)), (np.int64(404), np.int64(609)), (np.int64(1226), np.int64(484)), (np.int64(449), np.int64(414))]
    indices_1 = [(np.int64(597), np.int64(650)), (np.int64(131), np.int64(59)), (np.int64(104), np.int64(48)), (np.int64(256), np.int64(407)), (np.int64(607), np.int64(440))]

    selected_patch_0, selected_patch_1 = CS.getSample(hsi_, patch_size, sample_per_class, indices_0, indices_1)


i =0
half_patch = patch_size // 2
print(hsi_.img[indices_0[i][0]][indices_0[i][1]])
print(selected_patch_0[i][half_patch][half_patch])

print(hsi_.img[indices_1[i][0]][indices_1[i][1]])
print(selected_patch_1[i][half_patch][half_patch])

random: 1
generating random sample
hsi shape
(1243, 684, 224)
creating 5 Randomly chosen 0 indices:
creating 5 Randomly chosen 1 indices:
indices 0 used: [(np.int64(526), np.int64(187)), (np.int64(537), np.int64(71)), (np.int64(496), np.int64(222)), (np.int64(1200), np.int64(102)), (np.int64(1178), np.int64(413))]
indices 1 used: [(np.int64(174), np.int64(66)), (np.int64(382), np.int64(580)), (np.int64(1202), np.int64(171)), (np.int64(469), np.int64(254)), (np.int64(267), np.int64(228))]
[-195 -423  311  356  398  568  618  629  610  576  532  484  448  419
  388  356  341  312  286  256  229  208  191  163  141  130  122  110
  101   98   95   83   80   75   64   52   33   28   20  -21    2   17
   24  -13    5   14    9    5    0  -24  -15   -8   12    3    8    1
    0  -20  -26  -31  -31 -138 -118 -116  -25  -12   -6    0    1    2
    3    1    3    0    0   -3   -7   -7  -17  -47 -122 -107 -134 -144
  -53  -18  -11  -18  -18  -23  -15  -15   -4   -2   -4    0   -5    3
   17   10

In [5]:
i =4
half_patch = patch_size // 2
print(hsi_.img[indices_0[i][0]][indices_0[i][1]])
print(selected_patch_0[i][half_patch][half_patch])

print(hsi_.img[indices_1[i][0]][indices_1[i][1]])
print(selected_patch_1[i][half_patch][half_patch])

[-243 -361  383  431  450  591  657  656  619  584  543  491  452  413
  381  354  334  305  279  260  234  206  184  160  138  125  115  108
   97   94   90   83   81   74   70   60   46   38   34  -11    3   24
   28   -7   14   18   17    8    9  -18  -10    0   16   11   11    9
    7  -11  -20  -23  -20  -79  -93  -94  -22  -12    0    2    3    6
    7    3    3    4    5    0   -1   -2  -12  -36  -85  -72 -102  -99
  -44  -13   -5  -15   -6  -20   -8  -12    0    2    1   -3    0  -10
   -8    1   -6  -14   -9  -23  -41  -86  -49    0    0    0    0    0
    0    0    0    0    0  -21  -47  -50  -12   -1   -4   -3   -6    1
    1   -3   -2    0    2    3   -1    0    0   -2   -1   -4    3    4
    2   -2    2    0   -7    4   -4   -6  -11  -24  -39  -36  -20    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
   -1  -13  -15  -10   -4  -16  -31  -22   -3   -3   -3   -6   -7    0
   -3    3   -3   -6   -2   -5   -1    0   -2   -6   -1   -4    0    3
   -3 

In [6]:
print(indices_0)
print(indices_1)
indices = indices_0 +  indices_1

print(indices)
# Concatenating along axis 0
x_train = np.concatenate((selected_patch_0, selected_patch_1), )

y_train = np.array([])

gt = hsi_.gt
for indice in indices:
    # print(gt[indice[0]][indice[1]])
    y_train = np.append(y_train, gt[indice[0]][indice[1]])

count = np.count_nonzero(y_train == 0)  # Count elements equal to 0
print(f'number of element equal 0 {count}')

count = np.count_nonzero(y_train == 1)  # Count elements equal to 1
print(f'number of element equal 1 {count}')



# Print shape to verify
print(f"x_train shape: {x_train.shape}")  # Expected output: (10, 9, 9, 224)
print(f"y_train shape: {y_train.shape}") 



[(np.int64(526), np.int64(187)), (np.int64(537), np.int64(71)), (np.int64(496), np.int64(222)), (np.int64(1200), np.int64(102)), (np.int64(1178), np.int64(413))]
[(np.int64(174), np.int64(66)), (np.int64(382), np.int64(580)), (np.int64(1202), np.int64(171)), (np.int64(469), np.int64(254)), (np.int64(267), np.int64(228))]
[(np.int64(526), np.int64(187)), (np.int64(537), np.int64(71)), (np.int64(496), np.int64(222)), (np.int64(1200), np.int64(102)), (np.int64(1178), np.int64(413)), (np.int64(174), np.int64(66)), (np.int64(382), np.int64(580)), (np.int64(1202), np.int64(171)), (np.int64(469), np.int64(254)), (np.int64(267), np.int64(228))]
number of element equal 0 5
number of element equal 1 5
x_train shape: (10, 9, 9, 224)
y_train shape: (10,)


In [7]:
n_category = 2
band_size = 224
num_per_category_augment_1 = num_per_category_augment_1
num_per_category_augment_2 = num_per_category_augment_2

data_augment1, label_augment1 = aug.Augment_data(x_train, y_train, n_category, patch_size, band_size, num_per_category_augment_1)

data_augment2, label_augment2 = aug.Augment_data2(x_train, y_train, n_category, patch_size, band_size, num_per_category_augment_2)

print(f"hasil augmentasi 1 shape: {data_augment1.shape}")
print(f"label augmentai 1 shape: {label_augment1.shape}")

print(f"hasil augmentasi 2 shape: {data_augment2.shape}")
print(f"label augmentasi 2 shape: {label_augment2.shape}")

print(label_augment1)
print(label_augment2)

# Count occurrences of each unique element
counts1 = np.bincount(label_augment1)

# Print results
for i, count in enumerate(counts1):
    print(f"Element {i} occurs {count} times.")

counts2 = np.bincount(label_augment2)

# Print results
for i, count in enumerate(counts2):
    print(f"Element {i} occurs {count} times.")

print(label_augment1[3])

j:  20
hasil augmentasi 1 shape: (20, 9, 9, 224)
label augmentai 1 shape: (20,)
hasil augmentasi 2 shape: (20, 9, 9, 224)
label augmentasi 2 shape: (20,)
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
Element 0 occurs 10 times.
Element 1 occurs 10 times.
Element 0 occurs 10 times.
Element 1 occurs 10 times.
0


In [8]:
data_augment = np.concatenate((data_augment1, data_augment2))
label_augment = np.concatenate((label_augment1, label_augment2))

print(f"hasil augmentasi gabungan untuk training: {data_augment.shape}")
print(f"label augmentasi gabungan: {label_augment.shape}")

# print(label_augment)

# Count occurrences of each unique element
counts = np.bincount(label_augment)

# Print results
for i, count in enumerate(counts):
    print(f"Element {i} occurs {count} times.")

hasil augmentasi gabungan untuk training: (40, 9, 9, 224)
label augmentasi gabungan: (40,)
Element 0 occurs 20 times.
Element 1 occurs 20 times.


In [9]:
test = data_augment[0]
test = torch.tensor(test)
test = test.to(torch.float32)
test = test.unsqueeze(0)

input = test
input = input.permute(0, 3, 1, 2)

test2 = data_augment[1]
test2 = torch.tensor(test2)
test2 = test2.to(torch.float32)
test2 = test2.unsqueeze(0)

input2 = test2
input2 = input2.permute(0, 3, 1, 2)

print(f"input shape: {input.shape}")
print(f"input2 shape: {input2.shape}")

# Pass the input through the model
model.eval()
p1, p2, z1, z2  = model(input, input2)

print(p1)
print(p2)
print(z1)
print(z2)

input shape: torch.Size([1, 224, 9, 9])
input2 shape: torch.Size([1, 224, 9, 9])
tensor([[ 0.0270,  0.0315,  0.0116,  ..., -0.0291, -0.0339, -0.0541]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0271,  0.0315,  0.0118,  ..., -0.0293, -0.0341, -0.0542]],
       grad_fn=<AddmmBackward0>)
tensor([[0.0581, 0.0000, 0.0098,  ..., 0.0000, 0.0000, 0.0000]])
tensor([[0.0592, 0.0000, 0.0095,  ..., 0.0000, 0.0000, 0.0000]])


In [10]:
lr = 0.01
batch_size = 40

init_lr = lr * batch_size / 256

gpu = 0


print(model)


SimSiam(
  (pre_conv): Sequential(
    (0): Conv2d(224, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (fc): Linear(in_features=256, out_features=200704, bias=True)
  (encoder): VGG(
    (features): Sequential(
      (0): ReLU(inplace=True)
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ReLU(inplace=True)
      (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True

In [11]:
criterion = nn.CosineSimilarity(dim=1).cuda(gpu)
print(gpu)
optim_params = model.parameters()

momentum = 0.9
weight_decay = 1e-4

optimizer = torch.optim.SGD(optim_params, init_lr,
                                momentum=momentum,
                                weight_decay=weight_decay)

cudnn.benchmark = True
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

augmentation = [
    transforms.RandomHorizontalFlip(),  # Flip along width
    transforms.RandomVerticalFlip(),    # Flip along height
    transforms.RandomRotation(20),      # Rotate image slightly
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize hyperspectral data
]

transform = simsiam.loader.TwoCropsTransform(transforms.Compose(augmentation))

print(data_augment.shape)

0
(40, 9, 9, 224)


In [12]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, images, transform=None):
        """
        Args:
            images (Tensor or list of Tensors): Preloaded images of shape (N, 9, 9, 224)
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.images = images  # Assuming it's a list or tensor
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        
        if self.transform:
            img1 = self.transform(img)  # First augmentation
            img2 = self.transform(img)  # Second augmentation
        
            return img1, img2  # Return both augmented versions
        
        return img, img  # If no transform is provided, return the original image twice


# Example usage
preloaded_images = data_augment  # Example tensor with 100 images
X_train = torch.tensor(preloaded_images)
X_train = X_train.to(torch.float32)
X_train = X_train.permute(0, 3, 1, 2)
print(f"X_train shape: {X_train.shape}")

# Define transformations if needed
transform = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Example normalization
])

train_dataset = CustomDataset(X_train, transform=transform)

train_sampler = None

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=(train_sampler is None),
    num_workers=0,
    pin_memory=True,
    sampler=train_sampler,
    drop_last=True
)

# 7. Check Output

batch1, batch2 = next(iter(train_loader))

print(f"bacth size: {batch1.size()}")
print(f"length batch: {len(batch1)}")  # Should print 2 (Two transformed views per image)
print(f"{batch1[0][0].shape}")  # Should print torch.Size([9, 9, 224]) 


X_train shape: torch.Size([40, 224, 9, 9])
bacth size: torch.Size([40, 224, 9, 9])
length batch: 40
torch.Size([9, 9])


In [13]:
def adjust_learning_rate(optimizer, init_lr, epoch, epochs):
    """Decay the learning rate based on schedule"""
    cur_lr = init_lr * 0.5 * (1. + math.cos(math.pi * epoch / epochs))
    for param_group in optimizer.param_groups:
        if 'fix_lr' in param_group and param_group['fix_lr']:
            param_group['lr'] = init_lr
        else:
            param_group['lr'] = cur_lr

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
    
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [14]:
def train(train_loader, model, criterion, optimizer, epoch, device):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()

    for i, (images1, images2) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        input1 = images1
        input2 = images2

      
        input1 = input1.to(device, non_blocking=True)
        input2 = input2.to(device, non_blocking=True)
           

        p1, p2, z1, z2 = model(x1=input1, x2=input2) 
        loss = -(criterion(p1, z2).mean() + criterion(p2, z1).mean()) * 0.5

        losses.update(loss.item(), input1.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:
            progress.display(i)

In [15]:
start_epoch = 0
epochs = epochs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model.to(device)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')


for epoch in range(start_epoch, epochs):
    adjust_learning_rate(optimizer, init_lr, epoch, epochs)

    # train for one epoch

    train(train_loader, model, criterion, optimizer, epoch, device)

    if epoch + 1 <= 100:
        if epoch+1 == 5:
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': 'vgg16',
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict(),
            }, is_best=False, filename='models/pretrain/{}_checkpoint_{:04d}.pth.tar'.format(timestamp, epoch+1))
        elif (epoch+1) % 10 == 0:
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': 'vgg16',
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict(),
            }, is_best=False, filename='models/pretrain/{}_checkpoint_{:04d}.pth.tar'.format(timestamp, epoch+1))
    elif (epoch + 1 > 100):
        if (epoch + 1) % 25 == 0:
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': 'vgg16',
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict(),
            }, is_best=False, filename='models/pretrain/{}_checkpoint_{:04d}.pth.tar'.format(timestamp, epoch+1))

Epoch: [0][0/1]	Time  4.426 ( 4.426)	Data  0.023 ( 0.023)	Loss 0.0000 (0.0000)
Epoch: [1][0/1]	Time  0.357 ( 0.357)	Data  0.030 ( 0.030)	Loss -0.0020 (-0.0020)
Epoch: [2][0/1]	Time  0.198 ( 0.198)	Data  0.022 ( 0.022)	Loss 0.0027 (0.0027)
Epoch: [3][0/1]	Time  0.349 ( 0.349)	Data  0.022 ( 0.022)	Loss 0.0002 (0.0002)
Epoch: [4][0/1]	Time  0.380 ( 0.380)	Data  0.026 ( 0.026)	Loss -0.0008 (-0.0008)
Epoch: [5][0/1]	Time  0.190 ( 0.190)	Data  0.049 ( 0.049)	Loss -0.0008 (-0.0008)
Epoch: [6][0/1]	Time  0.443 ( 0.443)	Data  0.025 ( 0.025)	Loss 0.0006 (0.0006)
Epoch: [7][0/1]	Time  0.355 ( 0.355)	Data  0.015 ( 0.015)	Loss 0.0031 (0.0031)
Epoch: [8][0/1]	Time  0.360 ( 0.360)	Data  0.015 ( 0.015)	Loss 0.0013 (0.0013)
Epoch: [9][0/1]	Time  0.438 ( 0.438)	Data  0.014 ( 0.014)	Loss -0.0022 (-0.0022)
Epoch: [10][0/1]	Time  0.197 ( 0.197)	Data  0.042 ( 0.042)	Loss 0.0026 (0.0026)
Epoch: [11][0/1]	Time  0.375 ( 0.375)	Data  0.020 ( 0.020)	Loss -0.0016 (-0.0016)
Epoch: [12][0/1]	Time  0.356 ( 0.356)	Da